-sandbox
## Spark Crash Course


** What you will learn today:**
* How to Load/Extract the Data into Spark?
* How to Understand the Data using different Plots?
* Spark Structured and Unstructured APIs.
* Spark DataFrames : Transformations and Actions.
* Spark SQL intro.

Ref : https://spark.apache.org/docs/2.3.0/sql-programming-guide.html 

[DataFrame FAQs](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#dataframe-faqs)

## The Data

The dataset contains bike rental info from 2011 and 2012 in the Capital bikeshare system, plus additional relevant information such as weather.  

This dataset is from Fanaee-T and Gama (2013) and is hosted by the <a href="http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset" target="_blank">UCI Machine Learning Repository</a>.

In [0]:
display(dbutils.fs.ls("dbfs:/FileStore/tables/"))

path,name,size
dbfs:/FileStore/tables/allstate/,allstate/,0
dbfs:/FileStore/tables/clustering/,clustering/,0
dbfs:/FileStore/tables/day.csv,day.csv,57569
dbfs:/FileStore/tables/hour.csv,hour.csv,1156736
dbfs:/FileStore/tables/orangeTelecom/,orangeTelecom/,0
dbfs:/FileStore/tables/sample_linear_regression_data.txt,sample_linear_regression_data.txt,119069
dbfs:/FileStore/tables/stopwords.txt,stopwords.txt,2557
dbfs:/FileStore/tables/testing_bot_data.csv,testing_bot_data.csv,980391
dbfs:/FileStore/tables/training_bot_data.csv,training_bot_data.csv,5745947
dbfs:/FileStore/tables/tweet_data.csv,tweet_data.csv,227655107


### Spark APIs 

1. Create a Spark Context : Its required for executing Operations (Transform + Actions)
2. Create Spark Session : Driver Process to control Spark APIs
Link - https://spark.apache.org/docs/2.3.0/sql-programming-guide.html#starting-point-sparksession

1. Low Level Unstructured APIs
    * RDDS
2. Higher Level Structured APIs
    * DataFrame : Table of Data with Rows and Columns
    * SQL 
    * DataSets : TypeSafe 
    
Note - No performance difference between writing DataFrame Transformations or SQL queries, 
They both “compile” to the same underlying plan that we specify in DataFrame code

In [0]:
from pyspark.sql import SparkSession

spark

# spark = SparkSession \
#     .builder \
#     .appName("Python Spark SQL basic example") \
#     .config("spark.some.config.option", "some-value") \
#     .master("Resourse_manager") \ # (local/yarn)
#     .getOrCreate()

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

### DataFrame

Why DataFrame?

In Apache Spark , DataFrame is a distributed collection of Rows under Named Columns. (like Tables in SQL and Excel Sheet)
1. Immutable : we can Create DataFrames/RDDs but cannot change it.
     * DF1 --> Transformer --> DF2
     * Lineage can be tracked easily 
2. Lazy Evalution : Task will not be executed until an action is performed.
3. Distributed : distribued across the Cluster (partitions)
4. Resilient : replication of Partitions 

NOTE : 

* For implicit conversions like converting RDDs to DataFrames 
```import spark.implicits._```

* You can find the similar functionality/ transformations as in Pandas DataFrame.

BUT major differences are: 
1. Running tasks parallel on different nodes
2. Lazy evaluation 
3. DFs are immutable in Nature 
4. Pandas API support more (statistical) operations than PySpark DataFrame.

Spark ML uses DataFrame from Spark SQL as an ML dataset.

Dataframe of data types : 
```DataFrame [col_TEXT, 
           col_NUMERICAL, 
           col_ARRAY, 
           col_MAP, 
           col_FEATURE_VECTOR, 
           col_TRUE_LABELS, 
           col_PREDICTIONS
          ]```

#### You can also Load Data from
* Different DataFormats 
* * JSON, CSV, Parquet, ORC, Avro, LibSVM, Image
* Different DataStores
* * mySQL , Hbase, Hive, cassandra, MongoDB, Kafka, ElasticSearch

https://spark.apache.org/docs/latest/sql-data-sources.html

Example - 
* hdfs://FileStore/tables/
* s3://FileStore/tables/hour.csv
* dbfs://FileStore/tables/hour.csv

#### Create Dataframe from a file 
 
```
// from json file
spark.read.json(path_file.json)
// from text file 
spark.read.textFile("path_file.txt")
// from CSV file 
spark.read.csv("path_file.csv")    
    ```

In [0]:
fileName = "dbfs:/FileStore/tables/hour.csv"

initialDF = (spark.read          # Our DataFrameReader
  .option("header", "true")      # Let Spark know we have a header
  .option("inferSchema", "true") # Infering the schema (it is a small dataset)
  .csv(fileName)                 # Location of our data
  .cache()                       # Mark the DataFrame as cached.
)

initialDF.count()                # Materialize the cache

initialDF.printSchema()

root
-- instant: integer (nullable = true)
-- dteday: timestamp (nullable = true)
-- season: integer (nullable = true)
-- yr: integer (nullable = true)
-- mnth: integer (nullable = true)
-- hr: integer (nullable = true)
-- holiday: integer (nullable = true)
-- weekday: integer (nullable = true)
-- workingday: integer (nullable = true)
-- weathersit: integer (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- hum: double (nullable = true)
-- windspeed: double (nullable = true)
-- casual: integer (nullable = true)
-- registered: integer (nullable = true)
-- cnt: integer (nullable = true)

In [0]:
initialDF.explain()

== Physical Plan ==
InMemoryTableScan [instant#168, dteday#169, season#170, yr#171, mnth#172, hr#173, holiday#174, weekday#175, workingday#176, weathersit#177, temp#178, atemp#179, hum#180, windspeed#181, casual#182, registered#183, cnt#184]
 +- InMemoryRelation [instant#168, dteday#169, season#170, yr#171, mnth#172, hr#173, holiday#174, weekday#175, workingday#176, weathersit#177, temp#178, atemp#179, hum#180, windspeed#181, casual#182, registered#183, cnt#184], StorageLevel(disk, memory, deserialized, 1 replicas)
 +- *(1) FileScan csv [instant#168,dteday#169,season#170,yr#171,mnth#172,hr#173,holiday#174,weekday#175,workingday#176,weathersit#177,temp#178,atemp#179,hum#180,windspeed#181,casual#182,registered#183,cnt#184] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/hour.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<instant:int,dteday:timestamp,season:int,yr:int,mnth:int,hr:int,holiday:int,weekday:int,wor...

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
def weathersit_values(r):
    if r == 1 : return "Clear"
    elif r == 2 : return "Mist"
    elif r == 3 : return "Light Rain-Snow"
    else: return "Heavy Rain"

def season_values(r):
    if r == 1 : return "spring"
    elif r == 2 : return "summer"
    elif r == 3 : return "fall"
    else: return "winter"
    
    
weathersitTransorm = udf(weathersit_values, StringType())
seasonTransorm = udf(season_values, StringType())

newDF = initialDF.\
withColumn("weathersit",weathersitTransorm(initialDF.weathersit)).\
withColumn("season",seasonTransorm(initialDF.season))

### Display Data in different Styles

In [0]:
initialDF.printSchema()

root
-- instant: integer (nullable = true)
-- dteday: timestamp (nullable = true)
-- season: integer (nullable = true)
-- yr: integer (nullable = true)
-- mnth: integer (nullable = true)
-- hr: integer (nullable = true)
-- holiday: integer (nullable = true)
-- weekday: integer (nullable = true)
-- workingday: integer (nullable = true)
-- weathersit: integer (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- hum: double (nullable = true)
-- windspeed: double (nullable = true)
-- casual: integer (nullable = true)
-- registered: integer (nullable = true)
-- cnt: integer (nullable = true)

In [0]:
newDF.printSchema()

root
-- instant: integer (nullable = true)
-- dteday: timestamp (nullable = true)
-- season: string (nullable = true)
-- yr: integer (nullable = true)
-- mnth: integer (nullable = true)
-- hr: integer (nullable = true)
-- holiday: integer (nullable = true)
-- weekday: integer (nullable = true)
-- workingday: integer (nullable = true)
-- weathersit: string (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- hum: double (nullable = true)
-- windspeed: double (nullable = true)
-- casual: integer (nullable = true)
-- registered: integer (nullable = true)
-- cnt: integer (nullable = true)

In [0]:
# Display first Row
initialDF.first()

Out[64]: Row(instant=1, dteday=datetime.datetime(2011, 1, 1, 0, 0), season=1, yr=0, mnth=1, hr=0, holiday=0, weekday=6, workingday=0, weathersit=1, temp=0.24, atemp=0.2879, hum=0.81, windspeed=0.0, casual=3, registered=13, cnt=16)

In [0]:
# Display First 5 Rows
initialDF.head(5)

Out[9]: [Row(instant=1, dteday=datetime.datetime(2011, 1, 1, 0, 0), season=1, yr=0, mnth=1, hr=0, holiday=0, weekday=6, workingday=0, weathersit=1, temp=0.24, atemp=0.2879, hum=0.81, windspeed=0.0, casual=3, registered=13, cnt=16),
 Row(instant=2, dteday=datetime.datetime(2011, 1, 1, 0, 0), season=1, yr=0, mnth=1, hr=1, holiday=0, weekday=6, workingday=0, weathersit=1, temp=0.22, atemp=0.2727, hum=0.8, windspeed=0.0, casual=8, registered=32, cnt=40),
 Row(instant=3, dteday=datetime.datetime(2011, 1, 1, 0, 0), season=1, yr=0, mnth=1, hr=2, holiday=0, weekday=6, workingday=0, weathersit=1, temp=0.22, atemp=0.2727, hum=0.8, windspeed=0.0, casual=5, registered=27, cnt=32),
 Row(instant=4, dteday=datetime.datetime(2011, 1, 1, 0, 0), season=1, yr=0, mnth=1, hr=3, holiday=0, weekday=6, workingday=0, weathersit=1, temp=0.24, atemp=0.2879, hum=0.75, windspeed=0.0, casual=3, registered=10, cnt=13),
 Row(instant=5, dteday=datetime.datetime(2011, 1, 1, 0, 0), season=1, yr=0, mnth=1, hr=4, holiday=0, weekday=6, workingday=0, weathersit=1, temp=0.24, atemp=0.2879, hum=0.75, windspeed=0.0, casual=0, registered=1, cnt=1)]

In [0]:
# Results in Table format
initialDF.show(2,truncate= True)

+-------+-------------------+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+------+----------+---+
instant| dteday|season| yr|mnth| hr|holiday|weekday|workingday|weathersit|temp| atemp| hum|windspeed|casual|registered|cnt|
+-------+-------------------+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+------+----------+---+
 1|2011-01-01 00:00:00| 1| 0| 1| 0| 0| 6| 0| 1|0.24|0.2879|0.81| 0.0| 3| 13| 16|
 2|2011-01-01 00:00:00| 1| 0| 1| 1| 0| 6| 0| 1|0.22|0.2727| 0.8| 0.0| 8| 32| 40|
+-------+-------------------+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+------+----------+---+
only showing top 2 rows

#### Range

In [0]:
myRangeDF = spark.range(1000)
myRangeDF.show()
myRangeDF.show(4,truncate= True)

+---+
 id|
+---+
 0|
 1|
 2|
 3|
 4|
 5|
 6|
 7|
 8|
 9|
 10|
 11|
 12|
 13|
 14|
 15|
 16|
 17|
 18|
 19|
+---+
only showing top 20 rows

+---+
 id|
+---+
 0|
 1|
 2|
 3|
+---+
only showing top 4 rows

### Different Ways to Access Columns

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import StringType

#import org.apache.spark.sql.functions.{expr, col, column}

newDF.select(
  expr("season"),
  col("season"),
  column("season"))\
  .show(3)


+------+------+------+
season|season|season|
+------+------+------+
spring|spring|spring|
spring|spring|spring|
spring|spring|spring|
+------+------+------+
only showing top 3 rows

### How many Rows and Columns

In [0]:
# How many rows ?
newDF.count()

Out[13]: 17379

In [0]:
#  How many columns ?
len(newDF.columns)

Out[14]: 17

### Select Columns

In [0]:
# How to Select column(s)  
# first 5 rows 
newDF.select("atemp","temp","windspeed").show(5)

+------+----+---------+
 atemp|temp|windspeed|
+------+----+---------+
0.2879|0.24| 0.0|
0.2727|0.22| 0.0|
0.2727|0.22| 0.0|
0.2879|0.24| 0.0|
0.2879|0.24| 0.0|
+------+----+---------+
only showing top 5 rows

### Distinct numbers

In [0]:
# Distinct numbers
newDF.select("season").distinct().show()

# Count of Distinct Numbers
newDF.select("windspeed").distinct().count()

+------+
season|
+------+
winter|
summer|
spring|
 fall|
+------+

Out[16]: 30

In [0]:
# check df empty
newDF.rdd.isEmpty()

Out[17]: False

In [0]:
# convert a RDD to DF 
rdd_df = newDF.rdd

# convert DF back to RDD
rdd_df.toDF()

Out[18]: DataFrame[instant: bigint, dteday: timestamp, season: string, yr: bigint, mnth: bigint, hr: bigint, holiday: bigint, weekday: bigint, workingday: bigint, weathersit: string, temp: double, atemp: double, hum: double, windspeed: double, casual: bigint, registered: bigint, cnt: bigint]

### Filter Rows

In [0]:
springFilerDF = newDF.filter(newDF.season != "spring")
springFilerDF.show()

+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+
instant| dteday|season| yr|mnth| hr|holiday|weekday|workingday| weathersit|temp| atemp| hum|windspeed|casual|registered|cnt|
+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+
 1808|2011-03-21 00:00:00|summer| 0| 3| 0| 0| 1| 1|Light Rain-Snow|0.34| 0.303|0.66| 0.3881| 2| 11| 13|
 1809|2011-03-21 00:00:00|summer| 0| 3| 1| 0| 1| 1| Mist|0.34| 0.303|0.71| 0.3881| 1| 6| 7|
 1810|2011-03-21 00:00:00|summer| 0| 3| 2| 0| 1| 1| Mist|0.34| 0.303|0.71| 0.3284| 1| 5| 6|
 1811|2011-03-21 00:00:00|summer| 0| 3| 3| 0| 1| 1| Mist|0.34| 0.303|0.71| 0.3284| 0| 1| 1|
 1812|2011-03-21 00:00:00|summer| 0| 3| 5| 0| 1| 1| Clear|0.32| 0.303|0.81| 0.2985| 1| 1| 2|
 1813|2011-03-21 00:00:00|summer| 0| 3| 6| 0| 1| 1|Light Rain-Snow|0.32| 0.303|0.76| 0.2537| 2| 30| 32|
 1814|2011-03-21 00:00:00|summer| 0| 3| 7| 0| 1| 1|Light Rain-Snow| 0.3|0.2727|0.87| 0.4179| 3| 15| 18|
 1815|2011-03-21 00:00:00|summer| 0| 3| 8| 0| 1| 1| Mist| 0.3|0.2727|0.87| 0.4179| 3| 95| 98|
 1816|2011-03-21 00:00:00|summer| 0| 3| 9| 0| 1| 1| Mist|0.34| 0.303|0.81| 0.3284| 12| 115|127|
 1817|2011-03-21 00:00:00|summer| 0| 3| 10| 0| 1| 1| Mist|0.38|0.3939|0.76| 0.3881| 17| 45| 62|
 1818|2011-03-21 00:00:00|summer| 0| 3| 11| 0| 1| 1| Mist|0.42|0.4242|0.71| 0.2985| 32| 47| 79|
 1819|2011-03-21 00:00:00|summer| 0| 3| 12| 0| 1| 1| Mist|0.44|0.4394|0.77| 0.2537| 25| 56| 81|
 1820|2011-03-21 00:00:00|summer| 0| 3| 13| 0| 1| 1| Mist| 0.5|0.4848|0.63| 0.2239| 35| 77|112|
 1821|2011-03-21 00:00:00|summer| 0| 3| 14| 0| 1| 1| Mist|0.54|0.5152|0.64| 0.2239| 36| 65|101|
 1822|2011-03-21 00:00:00|summer| 0| 3| 15| 0| 1| 1| Mist|0.56|0.5303| 0.6| 0.2239| 41| 66|107|
 1823|2011-03-21 00:00:00|summer| 0| 3| 16| 0| 1| 1| Mist|0.54|0.5152|0.64| 0.2836| 29| 112|141|
 1824|2011-03-21 00:00:00|summer| 0| 3| 17| 0| 1| 1| Mist|0.54|0.5152|0.64| 0.2537| 41| 231|272|
 1825|2011-03-21 00:00:00|summer| 0| 3| 18| 0| 1| 1| Mist|0.52| 0.5|0.72| 0.2239| 44| 232|276|
 1826|2011-03-21 00:00:00|summer| 0| 3| 19| 0| 1| 1| Clear|0.58|0.5455| 0.6| 0.4179| 22| 199|221|
 1827|2011-03-21 00:00:00|summer| 0| 3| 20| 0| 1| 1| Clear|0.56|0.5303|0.64| 0.194| 16| 106|122|
+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+
only showing top 20 rows

In [0]:
newDF.filter(newDF.season=="summer") 
newDF.filter(newDF.season.contains("mmer")).show()
newDF.filter(newDF.season.like("%mmer")).show()

+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+
instant| dteday|season| yr|mnth| hr|holiday|weekday|workingday| weathersit|temp| atemp| hum|windspeed|casual|registered|cnt|
+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+
 1808|2011-03-21 00:00:00|summer| 0| 3| 0| 0| 1| 1|Light Rain-Snow|0.34| 0.303|0.66| 0.3881| 2| 11| 13|
 1809|2011-03-21 00:00:00|summer| 0| 3| 1| 0| 1| 1| Mist|0.34| 0.303|0.71| 0.3881| 1| 6| 7|
 1810|2011-03-21 00:00:00|summer| 0| 3| 2| 0| 1| 1| Mist|0.34| 0.303|0.71| 0.3284| 1| 5| 6|
 1811|2011-03-21 00:00:00|summer| 0| 3| 3| 0| 1| 1| Mist|0.34| 0.303|0.71| 0.3284| 0| 1| 1|
 1812|2011-03-21 00:00:00|summer| 0| 3| 5| 0| 1| 1| Clear|0.32| 0.303|0.81| 0.2985| 1| 1| 2|
 1813|2011-03-21 00:00:00|summer| 0| 3| 6| 0| 1| 1|Light Rain-Snow|0.32| 0.303|0.76| 0.2537| 2| 30| 32|
 1814|2011-03-21 00:00:00|summer| 0| 3| 7| 0| 1| 1|Light Rain-Snow| 0.3|0.2727|0.87| 0.4179| 3| 15| 18|
 1815|2011-03-21 00:00:00|summer| 0| 3| 8| 0| 1| 1| Mist| 0.3|0.2727|0.87| 0.4179| 3| 95| 98|
 1816|2011-03-21 00:00:00|summer| 0| 3| 9| 0| 1| 1| Mist|0.34| 0.303|0.81| 0.3284| 12| 115|127|
 1817|2011-03-21 00:00:00|summer| 0| 3| 10| 0| 1| 1| Mist|0.38|0.3939|0.76| 0.3881| 17| 45| 62|
 1818|2011-03-21 00:00:00|summer| 0| 3| 11| 0| 1| 1| Mist|0.42|0.4242|0.71| 0.2985| 32| 47| 79|
 1819|2011-03-21 00:00:00|summer| 0| 3| 12| 0| 1| 1| Mist|0.44|0.4394|0.77| 0.2537| 25| 56| 81|
 1820|2011-03-21 00:00:00|summer| 0| 3| 13| 0| 1| 1| Mist| 0.5|0.4848|0.63| 0.2239| 35| 77|112|
 1821|2011-03-21 00:00:00|summer| 0| 3| 14| 0| 1| 1| Mist|0.54|0.5152|0.64| 0.2239| 36| 65|101|
 1822|2011-03-21 00:00:00|summer| 0| 3| 15| 0| 1| 1| Mist|0.56|0.5303| 0.6| 0.2239| 41| 66|107|
 1823|2011-03-21 00:00:00|summer| 0| 3| 16| 0| 1| 1| Mist|0.54|0.5152|0.64| 0.2836| 29| 112|141|
 1824|2011-03-21 00:00:00|summer| 0| 3| 17| 0| 1| 1| Mist|0.54|0.5152|0.64| 0.2537| 41| 231|272|
 1825|2011-03-21 00:00:00|summer| 0| 3| 18| 0| 1| 1| Mist|0.52| 0.5|0.72| 0.2239| 44| 232|276|
 1826|2011-03-21 00:00:00|summer| 0| 3| 19| 0| 1| 1| Clear|0.58|0.5455| 0.6| 0.4179| 22| 199|221|
 1827|2011-03-21 00:00:00|summer| 0| 3| 20| 0| 1| 1| Clear|0.56|0.5303|0.64| 0.194| 16| 106|122|
+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+
only showing top 20 rows

+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+
instant| dteday|season| yr|mnth| hr|holiday|weekday|workingday| weathersit|temp| atemp| hum|windspeed|casual|registered|cnt|
+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+
 1808|2011-03-21 00:00:00|summer| 0| 3| 0| 0| 1| 1|Light Rain-Snow|0.34| 0.303|0.66| 0.3881| 2| 11| 13|
 1809|2011-03-21 00:00:00|summer| 0| 3| 1| 0| 1| 1| Mist|0.34| 0.303|0.71| 0.3881| 1| 6| 7|
 1810|2011-03-21 00:00:00|summer| 0| 3| 2| 0| 1| 1| Mist|0.34| 0.303|0.71| 0.3284| 1| 5| 6|
 1811|2011-03-21 00:00:00|summer| 0| 3| 3| 0| 1| 1| Mist|0.34| 0.303|0.71| 0.3284| 0| 1| 1|
 1812|2011-03-21 00:00:00|summer| 0| 3| 5| 0| 1| 1| Clear|0.32| 0.303|0.81| 0.2985| 1| 1| 2|
 1813|2011-03-21 00:00:00|summer| 0| 3| 6| 0| 1| 1|Light Rain-Snow|0.32| 0.303|0.76| 0.2537| 2| 30| 32|
 1814|2011-03-21 00:00:00|summer| 0| 3| 7| 0| 1| 1|Light Rain-Snow| 0.3|0.2727|0.87| 0.4179| 3| 15| 18|
 1815|2011-03-21 00:00:00|summer| 0| 3| 8| 0| 1| 1| Mist| 0.3|0.2727|0.87| 0.4179| 3| 95| 98|
 1816|2011-03-21 00:00:00|summer| 0| 3| 9| 0| 1| 1| Mist|0.34| 0.303|0.81| 0.3284| 12| 115|127|
 1817|2011-03-21 00:00:00|summer| 0| 3| 10| 0| 1| 1| Mist|0.38|0.3939|0.76| 0.3881| 17| 45| 62|
 1818|2011-03-21 00:00:00|summer| 0| 3| 11| 0| 1| 1| Mist|0.42|0.4242|0.71| 0.2985| 32| 47| 79|
 1819|2011-03-21 00:

### Type CASTing 

Syntax - dataFrame["columnName"].cast(DataType())

In [0]:
newDF.schema
# newDF.printSchema()

Out[72]: StructType(List(StructField(instant,IntegerType,true),StructField(dteday,TimestampType,true),StructField(season,StringType,true),StructField(yr,IntegerType,true),StructField(mnth,IntegerType,true),StructField(hr,IntegerType,true),StructField(holiday,IntegerType,true),StructField(weekday,IntegerType,true),StructField(workingday,IntegerType,true),StructField(weathersit,StringType,true),StructField(temp,DoubleType,true),StructField(atemp,DoubleType,true),StructField(hum,DoubleType,true),StructField(windspeed,DoubleType,true),StructField(casual,IntegerType,true),StructField(registered,IntegerType,true),StructField(cnt,IntegerType,true)))

In [0]:
newDF.select("cnt").schema

Out[22]: StructType(List(StructField(cnt,IntegerType,true)))

In [0]:
from pyspark.sql.types import DoubleType

totalCnt = col("cnt")
newDF.select(totalCnt.cast(DoubleType()))

Out[73]: DataFrame[cnt: double]

### Group By

In [0]:
newDF.groupby('season').count().show()

+------+-----+
season|count|
+------+-----+
winter| 4232|
summer| 4409|
spring| 4242|
 fall| 4496|
+------+-----+

In [0]:
count_seasons = newDF.groupby('season').count()
count_seasons.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[season#7244], functions=[finalmerge_count(merge count#8247L) AS count(1)#8157L])
+- Exchange hashpartitioning(season#7244, 200), [id=#2786]
 +- *(1) HashAggregate(keys=[season#7244], functions=[partial_count(1) AS count#8247L])
 +- *(1) Project [pythonUDF0#8248 AS season#7244]
 +- BatchEvalPython [season_values(season#7000)], [pythonUDF0#8248]
 +- InMemoryTableScan [season#7000]
 +- InMemoryRelation [instant#6998, dteday#6999, season#7000, yr#7001, mnth#7002, hr#7003, holiday#7004, weekday#7005, workingday#7006, weathersit#7007, temp#7008, atemp#7009, hum#7010, windspeed#7011, casual#7012, registered#7013, cnt#7014], StorageLevel(disk, memory, deserialized, 1 replicas)
 +- *(1) FileScan csv [instant#168,dteday#169,season#170,yr#171,mnth#172,hr#173,holiday#174,weekday#175,workingday#176,weathersit#177,temp#178,atemp#179,hum#180,windspeed#181,casual#182,registered#183,cnt#184] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/hour.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<instant:int,dteday:timestamp,season:int,yr:int,mnth:int,hr:int,holiday:int,weekday:int,wor...

### Order By

In [0]:
newDF.orderBy(newDF.cnt.desc()).show(5)
newDF.orderBy(newDF.casual.desc(),newDF.cnt ).printSchema()

+-------+-------------------+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+------+----------+---+
instant| dteday|season| yr|mnth| hr|holiday|weekday|workingday|weathersit|temp| atemp| hum|windspeed|casual|registered|cnt|
+-------+-------------------+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+------+----------+---+
 14774|2012-09-12 00:00:00| fall| 1| 9| 18| 0| 3| 1| Clear|0.66|0.6212|0.44| 0.2537| 91| 886|977|
 14965|2012-09-20 00:00:00| fall| 1| 9| 17| 0| 4| 1| Clear|0.64|0.6212| 0.5| 0.2239| 91| 885|976|
 14749|2012-09-11 00:00:00| fall| 1| 9| 17| 0| 2| 1| Clear| 0.7|0.6364|0.28| 0.0| 168| 802|970|
 14726|2012-09-10 00:00:00| fall| 1| 9| 18| 0| 1| 1| Clear|0.62|0.6212|0.35| 0.2985| 111| 857|968|
 15085|2012-09-25 00:00:00|winter| 1| 9| 17| 0| 2| 1| Clear|0.66|0.6212|0.39| 0.2836| 107| 860|967|
+-------+-------------------+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+------+----------+---+
only showing top 5 rows

root
-- instant: integer (nullable = true)
-- dteday: timestamp (nullable = true)
-- season: string (nullable = true)
-- yr: integer (nullable = true)
-- mnth: integer (nullable = true)
-- hr: integer (nullable = true)
-- holiday: integer (nullable = true)
-- weekday: integer (nullable = true)
-- workingday: integer (nullable = true)
-- weathersit: string (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- hum: double (nullable = true)
-- windspeed: double (nullable = true)
-- casual: integer (nullable = true)
-- registered: integer (nullable = true)
-- cnt: integer (nullable = true)

### Add New Column or Replace

In [0]:
# The withColumn operation will take 2 parameters.
###  Column name which we want add /replace.
###  Expression on column.

* **hum**: Normalized humidity. The values are divided to 100 (max)
* **windspeed**: Normalized wind speed. The values are divided to 67 (max)

In [0]:
hum_new_DF = newDF.withColumn("hum_new",newDF.hum * 100)
hum_new_DF.printSchema()
hum_new_DF.show()

root
-- instant: integer (nullable = true)
-- dteday: timestamp (nullable = true)
-- season: string (nullable = true)
-- yr: integer (nullable = true)
-- mnth: integer (nullable = true)
-- hr: integer (nullable = true)
-- holiday: integer (nullable = true)
-- weekday: integer (nullable = true)
-- workingday: integer (nullable = true)
-- weathersit: string (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- hum: double (nullable = true)
-- windspeed: double (nullable = true)
-- casual: integer (nullable = true)
-- registered: integer (nullable = true)
-- cnt: integer (nullable = true)
-- hum_new: double (nullable = true)

+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+-------+
instant| dteday|season| yr|mnth| hr|holiday|weekday|workingday| weathersit|temp| atemp| hum|windspeed|casual|registered|cnt|hum_new|
+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+-------+
 1|2011-01-01 00:00:00|spring| 0| 1| 0| 0| 6| 0| Clear|0.24|0.2879|0.81| 0.0| 3| 13| 16| 81.0|
 2|2011-01-01 00:00:00|spring| 0| 1| 1| 0| 6| 0| Clear|0.22|0.2727| 0.8| 0.0| 8| 32| 40| 80.0|
 3|2011-01-01 00:00:00|spring| 0| 1| 2| 0| 6| 0| Clear|0.22|0.2727| 0.8| 0.0| 5| 27| 32| 80.0|
 4|2011-01-01 00:00:00|spring| 0| 1| 3| 0| 6| 0| Clear|0.24|0.2879|0.75| 0.0| 3| 10| 13| 75.0|
 5|2011-01-01 00:00:00|spring| 0| 1| 4| 0| 6| 0| Clear|0.24|0.2879|0.75| 0.0| 0| 1| 1| 75.0|
 6|2011-01-01 00:00:00|spring| 0| 1| 5| 0| 6| 0| Mist|0.24|0.2576|0.75| 0.0896| 0| 1| 1| 75.0|
 7|2011-01-01 00:00:00|spring| 0| 1| 6| 0| 6| 0| Clear|0.22|0.2727| 0.8| 0.0| 2| 0| 2| 80.0|
 8|2011-01-01 00:00:00|spring| 0| 1| 7| 0| 6| 0| Clear| 0.2|0.2576|0.86| 0.0| 1| 2| 3| 86.0|
 9|2011-01-01 00:00:00|spring| 0| 1| 8| 0| 6| 0| Clear|0.24|0.2879|0.75| 0.0| 1| 7| 8| 75.0|
 10|2011-01-01 00:00:00|spring| 0| 1| 9| 0| 6| 0| Clear|0.32|0.3485|0.76| 0.0| 8| 6| 14| 76.0|
 11|2011-01-01 00:00:00|spring| 0| 1| 10| 0| 6| 0| Clear|0.38|0.3939|0.76| 0.2537| 12| 24| 36| 76.0|
 12|2011-01-01 00:00:00|spring| 0| 1| 11| 0| 6| 0| Clear|0.36|0.3333|0.81| 0.2836| 26| 30| 56| 81.0|
 13|2011-01-01 00:00:00|spring| 0| 1| 12| 0| 6| 0| Clear|0.42|0.4242|0.77| 0.2836| 29| 55| 84| 77.0|
 14|2011-01-01 00:00:00|spring| 0| 1| 13| 0| 6| 0| Mist|0.46|0.4545|0.72| 0.2985| 47| 47| 94| 72.0|
 15|2011-01-01 00:00:00|spring| 0| 1| 14| 0| 6| 0| Mist|0.46|0.4545|0.72| 0.2836| 35| 71|106| 72.0|
 16|2011-01-01 00:00:00|spring| 0| 1| 15| 0| 6| 0| Mist|0.44|0.4394|0.77| 0.2985| 40| 70|110| 77.0|
 17|2011-01-01 00:00:00|spring| 0| 1| 16| 0| 6| 0| Mist|0.42|0.4242|0.82| 0.2985| 41| 52| 93| 82.0|
 18|2011-01-01 00:00:00|spring| 0| 1| 17| 0| 6| 0| Mist|0.44|0.4394|0.82| 0.2836| 15| 52| 67| 82.0|
 19|2011-01-01 00:00:00|spring| 0| 1| 18| 0| 6| 0|Light Rain-Snow|0.42|0.4242|0.88| 0.2537| 9| 26| 35| 88.0|
 20|2011-01-01 00:00:00|spring| 0| 1| 19| 0| 6| 0|Light Rain-Snow|0.42|0.4242|0.88| 0.2537| 6| 31| 37| 88.0|
+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+-------+
only showing top 20 rows

In [0]:
windspeed_new_DF = newDF.withColumn("windspeed", newDF.windspeed * 67)
windspeed_new_DF.printSchema()
windspeed_new_DF.select("windspeed").show()

root
-- instant: integer (nullable = true)
-- dteday: timestamp (nullable = true)
-- season: string (nullable = true)
-- yr: integer (nullable = true)
-- mnth: integer (nullable = true)
-- hr: integer (nullable = true)
-- holiday: integer (nullable = true)
-- weekday: integer (nullable = true)
-- workingday: integer (nullable = true)
-- weathersit: string (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- hum: double (nullable = true)
-- windspeed: double (nullable = true)
-- casual: integer (nullable = true)
-- registered: integer (nullable = true)
-- cnt: integer (nullable = true)

+------------------+
 windspeed|
+------------------+
 0.0|
 0.0|
 0.0|
 0.0|
 0.0|
 6.0032|
 0.0|
 0.0|
 0.0|
 0.0|
16.997899999999998|
 19.0012|
 19.0012|
19.999499999999998|
 19.0012|
19.999499999999998|
19.999499999999998|
 19.0012|
16.997899999999998|
16.997899999999998|
+------------------+
only showing top 20 rows

In [0]:
# Spark SQL functions lit() and typedLit() are used to add a new column by assigning a literal or constant value to Spark DataFrame. Both functions return Column as return type.

# Both of these are available in Spark by importing org.apache.spark.sql.functions

from pyspark.sql.functions import lit

## New Column - Replace a Column with Constants
newDF.withColumn("cnt", lit(10)).show()

## Add Integer to a column
newDF.select('*', (newDF.cnt + 10000).alias('newCnt')).show()

+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+
instant| dteday|season| yr|mnth| hr|holiday|weekday|workingday| weathersit|temp| atemp| hum|windspeed|casual|registered|cnt|
+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+
 1|2011-01-01 00:00:00|spring| 0| 1| 0| 0| 6| 0| Clear|0.24|0.2879|0.81| 0.0| 3| 13| 10|
 2|2011-01-01 00:00:00|spring| 0| 1| 1| 0| 6| 0| Clear|0.22|0.2727| 0.8| 0.0| 8| 32| 10|
 3|2011-01-01 00:00:00|spring| 0| 1| 2| 0| 6| 0| Clear|0.22|0.2727| 0.8| 0.0| 5| 27| 10|
 4|2011-01-01 00:00:00|spring| 0| 1| 3| 0| 6| 0| Clear|0.24|0.2879|0.75| 0.0| 3| 10| 10|
 5|2011-01-01 00:00:00|spring| 0| 1| 4| 0| 6| 0| Clear|0.24|0.2879|0.75| 0.0| 0| 1| 10|
 6|2011-01-01 00:00:00|spring| 0| 1| 5| 0| 6| 0| Mist|0.24|0.2576|0.75| 0.0896| 0| 1| 10|
 7|2011-01-01 00:00:00|spring| 0| 1| 6| 0| 6| 0| Clear|0.22|0.2727| 0.8| 0.0| 2| 0| 10|
 8|2011-01-01 00:00:00|spring| 0| 1| 7| 0| 6| 0| Clear| 0.2|0.2576|0.86| 0.0| 1| 2| 10|
 9|2011-01-01 00:00:00|spring| 0| 1| 8| 0| 6| 0| Clear|0.24|0.2879|0.75| 0.0| 1| 7| 10|
 10|2011-01-01 00:00:00|spring| 0| 1| 9| 0| 6| 0| Clear|0.32|0.3485|0.76| 0.0| 8| 6| 10|
 11|2011-01-01 00:00:00|spring| 0| 1| 10| 0| 6| 0| Clear|0.38|0.3939|0.76| 0.2537| 12| 24| 10|
 12|2011-01-01 00:00:00|spring| 0| 1| 11| 0| 6| 0| Clear|0.36|0.3333|0.81| 0.2836| 26| 30| 10|
 13|2011-01-01 00:00:00|spring| 0| 1| 12| 0| 6| 0| Clear|0.42|0.4242|0.77| 0.2836| 29| 55| 10|
 14|2011-01-01 00:00:00|spring| 0| 1| 13| 0| 6| 0| Mist|0.46|0.4545|0.72| 0.2985| 47| 47| 10|
 15|2011-01-01 00:00:00|spring| 0| 1| 14| 0| 6| 0| Mist|0.46|0.4545|0.72| 0.2836| 35| 71| 10|
 16|2011-01-01 00:00:00|spring| 0| 1| 15| 0| 6| 0| Mist|0.44|0.4394|0.77| 0.2985| 40| 70| 10|
 17|2011-01-01 00:00:00|spring| 0| 1| 16| 0| 6| 0| Mist|0.42|0.4242|0.82| 0.2985| 41| 52| 10|
 18|2011-01-01 00:00:00|spring| 0| 1| 17| 0| 6| 0| Mist|0.44|0.4394|0.82| 0.2836| 15| 52| 10|
 19|2011-01-01 00:00:00|spring| 0| 1| 18| 0| 6| 0|Light Rain-Snow|0.42|0.4242|0.88| 0.2537| 9| 26| 10|
 20|2011-01-01 00:00:00|spring| 0| 1| 19| 0| 6| 0|Light Rain-Snow|0.42|0.4242|0.88| 0.2537| 6| 31| 10|
+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+
only showing top 20 rows

+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+------+
instant| dteday|season| yr|mnth| hr|holiday|weekday|workingday| weathersit|temp| atemp| hum|windspeed|casual|registered|cnt|newCnt|
+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+------+
 1|2011-01-01 00:00:00|spring| 0| 1| 0| 0| 6| 0| Clear|0.24|0.2879|0.81| 0.0| 3| 13| 16| 10016|
 2|2011-01-01 00:00:00|spring| 0| 1| 1| 0| 6| 0| Clear|0.22|0.2727| 0.8| 0.0| 8| 32| 40| 10040|
 3|2011-01-01 00:00:00|spring| 0| 1| 2| 0| 6| 0| Clear|0.22|0.2727| 0.8| 0.0| 5| 27| 32| 10032|
 4|2011-01-01 00:00:00|spring| 0| 1| 3| 0| 6| 0| Clear|0.24|0.2879|0.75| 0.0| 3| 10| 13| 10013|
 5|2011-01-01 00:00:00|spring| 0| 1| 4| 0| 6| 0| Clear|0.24|0.2879|0.75| 0.0| 0| 1| 1| 10001|
 6|2011-01-01 00:00:00|spring| 0| 1| 5| 0| 6| 0| Mist|0.24|0.2576|0.75| 0.0896| 0| 1| 1| 10001|
 7|2011-01-01 00:00:00|spring| 0| 1| 6| 0| 6| 0| Clear|0.22|0.2727| 0.8| 0.0| 2| 0| 2| 10002|
 8|2011-01-01 00:00:00|spring| 0| 1| 7| 0| 6| 0| Clear| 0.2|0.2576|0.86| 0.0| 1| 2| 3| 10003|
 9|2011-01-01 00:00:00|spring| 0| 1| 8| 0| 6| 0| Clear|0.24|0.2879|0.75| 0.0| 1| 7| 8| 10008|
 10|2011-01-01 00:00:00|spring| 0| 1| 9| 0| 6| 0| Clear|0.32|0.3485|0.76| 0.0| 8| 6| 14| 10014|
 11|2011-01-01 00:00:00|spring| 0| 1| 10| 0| 6| 0| Clear|0.38|0.3939|0.76| 0.2537| 12| 24| 36| 10036|
 12|2011-01-01 00:00:00|spring| 0| 1| 11| 0| 6| 0| Clear|0.36|0.3333|0.81| 0.2836| 26| 30| 

In [0]:
newDF.select(col("cnt").alias("totalCnt")).show()
newDF.withColumnRenamed("cnt", "totalCnt").show()

+--------+
totalCnt|
+--------+
 16|
 40|
 32|
 13|
 1|
 1|
 2|
 3|
 8|
 14|
 36|
 56|
 84|
 94|
 106|
 110|
 93|
 67|
 35|
 37|
+--------+
only showing top 20 rows

+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+--------+
instant| dteday|season| yr|mnth| hr|holiday|weekday|workingday| weathersit|temp| atemp| hum|windspeed|casual|registered|totalCnt|
+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+--------+
 1|2011-01-01 00:00:00|spring| 0| 1| 0| 0| 6| 0| Clear|0.24|0.2879|0.81| 0.0| 3| 13| 16|
 2|2011-01-01 00:00:00|spring| 0| 1| 1| 0| 6| 0| Clear|0.22|0.2727| 0.8| 0.0| 8| 32| 40|
 3|2011-01-01 00:00:00|spring| 0| 1| 2| 0| 6| 0| Clear|0.22|0.2727| 0.8| 0.0| 5| 27| 32|
 4|2011-01-01 00:00:00|spring| 0| 1| 3| 0| 6| 0| Clear|0.24|0.2879|0.75| 0.0| 3| 10| 13|
 5|2011-01-01 00:00:00|spring| 0| 1| 4| 0| 6| 0| Clear|0.24|0.2879|0.75| 0.0| 0| 1| 1|
 6|2011-01-01 00:00:00|spring| 0| 1| 5| 0| 6| 0| Mist|0.24|0.2576|0.75| 0.0896| 0| 1| 1|
 7|2011-01-01 00:00:00|spring| 0| 1| 6| 0| 6| 0| Clear|0.22|0.2727| 0.8| 0.0| 2| 0| 2|
 8|2011-01-01 00:00:00|spring| 0| 1| 7| 0| 6| 0| Clear| 0.2|0.2576|0.86| 0.0| 1| 2| 3|
 9|2011-01-01 00:00:00|spring| 0| 1| 8| 0| 6| 0| Clear|0.24|0.2879|0.75| 0.0| 1| 7| 8|
 10|2011-01-01 00:00:00|spring| 0| 1| 9| 0| 6| 0| Clear|0.32|0.3485|0.76| 0.0| 8| 6| 14|
 11|2011-01-01 00:00:00|spring| 0| 1| 10| 0| 6| 0| Clear|0.38|0.3939|0.76| 0.2537| 12| 24| 36|
 12|2011-01-01 00:00:00|spring| 0| 1| 11| 0| 6| 0| Clear|0.36|0.3333|0.81| 0.2836| 26| 30| 56|
 13|2011-01-01 00:00:00|spring| 0| 1| 12| 0| 6| 0| Clear|0.42|0.4242|0.77| 0.2836| 29| 55| 84|
 14|2011-01-01 00:00:00|spring| 0| 1| 13| 0| 6| 0| Mist|0.46|0.4545|0.72| 0.2985| 47| 47| 94|
 15|2011-01-01 00:00:00|spring| 0| 1| 14| 0| 6| 0| Mist|0.46|0.4545|0.72| 0.2836| 35| 71| 106|
 16|2011-01-01 00:00:00|spring| 0| 1| 15| 0| 6| 0| Mist|0.44|0.4394|0.77| 0.2985| 40| 70| 110|
 17|2011-01-01 00:00:00|spring| 0| 1| 16| 0| 6| 0| Mist|0.42|0.4242|0.82| 0.2985| 41| 52| 93|
 18|2011-01-01 00:00:00|spring| 0| 1| 17| 0| 6| 0| Mist|0.44|0.4394|0.82| 0.2836| 15| 52| 67|
 19|2011-01-01 00:00:00|spring| 0| 1| 18| 0| 6| 0|Light Rain-Snow|0.42|0.4242|0.88| 0.2537| 9| 26| 35|
 20|2011-01-01 00:00:00|spring| 0| 1| 19| 0| 6| 0|Light Rain-Snow|0.42|0.4242|0.88| 0.2537| 6| 31| 37|
+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+--------+
only showing top 20 rows

### Statistical functions

Summary Stats 
* Mean, 
* StandardDeviance, 
* Min ,
* Max , 
* Count

For more [Statistical-and-mathematical-functions](https://databricks.com/blog/2015/06/02/statistical-and-mathematical-functions-with-dataframes-in-spark.html)


NOTE : 
1. Summary statistics (Mean, Standard Deviance, Min ,Max , Count) of Categorical columns
2. DataFrame with Categorical Columns (StringType) on Describe : 
    -  output : mean, stddev are NULL
    -   min & max values are calculated based on ASCII value of categories

In [0]:
# Summary statistics (Mean, StandardDeviance, Min ,Max , Count) of Numerical columns
# initialDF.describe().show(5)
initialDF.select("atemp","temp","windspeed").describe().show()

+-------+------------------+-------------------+-------------------+
summary| atemp| temp| windspeed|
+-------+------------------+-------------------+-------------------+
 count| 17379| 17379| 17379|
 mean|0.4757751021347581| 0.4969871684216586| 0.1900976063064631|
 stddev|0.1718502156353594|0.19255612124972202|0.12234022857279034|
 min| 0.0| 0.02| 0.0|
 max| 1.0| 1.0| 0.8507|
+-------+------------------+-------------------+-------------------+

### Pair wise frequency of categorical columns?

1. First column of each row will be the Distinct values of Season and Column names will be distinct values of Weather Situation
2. Note : Pair with no occurrences will have zero count in contingency table.

In [0]:
# Calculate pair wise frequency of categorical columns?
newDF.crosstab('weathersit','season').show()

+-----------------+----+------+------+------+
weathersit_season|fall|spring|summer|winter|
+-----------------+----+------+------+------+
 Light Rain-Snow| 269| 369| 406| 375|
 Heavy Rain| 0| 3| 0| 0|
 Mist| 947| 1205| 1144| 1248|
 Clear|3280| 2665| 2859| 2609|
+-----------------+----+------+------+------+

In [0]:
newDF.stat.freqItems(["season"],0.4).show(truncate= False)
# df.stat.approxQuantile(...)
# df.stat.bloomFilter(...)
# df.stat.countMinSketch()

+----------------+
season_freqItems|
+----------------+
[winter, fall] |
+----------------+

### SQL Queries 

1. Register the DataFrame as a Table 
2. Use spark session.sql function 
3. Returns a new DataFrame

In [0]:
newDF.createOrReplaceTempView('Bike_Prediction_Table')
# initialDF.createOrReplaceTempView('BIKE_PRedictions_OLD')

In [0]:
sqlContext

Out[82]: <pyspark.sql.context.HiveContext at 0x7fda3f2a1ac8>

In [0]:
sqlContext.sql("select season, cnt from Bike_Prediction_Table ").show()
# newDF.select("season","cnt")

+------+---+
season|cnt|
+------+---+
spring| 16|
spring| 40|
spring| 32|
spring| 13|
spring| 1|
spring| 1|
spring| 2|
spring| 3|
spring| 8|
spring| 14|
spring| 36|
spring| 56|
spring| 84|
spring| 94|
spring|106|
spring|110|
spring| 93|
spring| 67|
spring| 35|
spring| 37|
+------+---+
only showing top 20 rows

In [0]:
sqlContext.sql("select * from Bike_Prediction_Table where season='summer' ").show(5)

+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+-----+----+---------+------+----------+---+
instant| dteday|season| yr|mnth| hr|holiday|weekday|workingday| weathersit|temp|atemp| hum|windspeed|casual|registered|cnt|
+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+-----+----+---------+------+----------+---+
 1808|2011-03-21 00:00:00|summer| 0| 3| 0| 0| 1| 1|Light Rain-Snow|0.34|0.303|0.66| 0.3881| 2| 11| 13|
 1809|2011-03-21 00:00:00|summer| 0| 3| 1| 0| 1| 1| Mist|0.34|0.303|0.71| 0.3881| 1| 6| 7|
 1810|2011-03-21 00:00:00|summer| 0| 3| 2| 0| 1| 1| Mist|0.34|0.303|0.71| 0.3284| 1| 5| 6|
 1811|2011-03-21 00:00:00|summer| 0| 3| 3| 0| 1| 1| Mist|0.34|0.303|0.71| 0.3284| 0| 1| 1|
 1812|2011-03-21 00:00:00|summer| 0| 3| 5| 0| 1| 1| Clear|0.32|0.303|0.81| 0.2985| 1| 1| 2|
+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+-----+----+---------+------+----------+---+
only showing top 5 rows

In [0]:
sqlContext.sql("select * from Bike_Prediction_Table where season='summer' and windspeed > 0.4").show()

+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+
instant| dteday|season| yr|mnth| hr|holiday|weekday|workingday| weathersit|temp| atemp| hum|windspeed|casual|registered|cnt|
+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+
 1814|2011-03-21 00:00:00|summer| 0| 3| 7| 0| 1| 1|Light Rain-Snow| 0.3|0.2727|0.87| 0.4179| 3| 15| 18|
 1815|2011-03-21 00:00:00|summer| 0| 3| 8| 0| 1| 1| Mist| 0.3|0.2727|0.87| 0.4179| 3| 95| 98|
 1826|2011-03-21 00:00:00|summer| 0| 3| 19| 0| 1| 1| Clear|0.58|0.5455| 0.6| 0.4179| 22| 199|221|
 1840|2011-03-22 00:00:00|summer| 0| 3| 9| 0| 2| 1| Mist| 0.4|0.4091|0.58| 0.4627| 19| 159|178|
 1874|2011-03-23 00:00:00|summer| 0| 3| 20| 0| 3| 1|Light Rain-Snow|0.36| 0.303|0.87| 0.6418| 3| 72| 75|
 1875|2011-03-23 00:00:00|summer| 0| 3| 21| 0| 3| 1|Light Rain-Snow|0.32|0.2879| 1.0| 0.4179| 0| 22| 22|
 2083|2011-04-01 00:00:00|summer| 0| 4| 15| 0| 5| 1| Clear|0.34| 0.303|0.46| 0.4179| 30| 81|111|
 2084|2011-04-01 00:00:00|summer| 0| 4| 16| 0| 5| 1| Clear|0.34| 0.303|0.46| 0.4179| 42| 125|167|
 2125|2011-04-03 00:00:00|summer| 0| 4| 9| 0| 0| 0| Clear|0.36|0.3182| 0.5| 0.4925| 68| 74|142|
 2126|2011-04-03 00:00:00|summer| 0| 4| 10| 0| 0| 0| Clear| 0.4|0.4091|0.43| 0.4179| 111| 104|215|
 2152|2011-04-04 00:00:00|summer| 0| 4| 12| 0| 1| 1| Clear|0.64|0.6212|0.36| 0.4627| 59| 98|157|
 2154|2011-04-04 00:00:00|summer| 0| 4| 14| 0| 1| 1| Mist|0.74|0.6515|0.27| 0.4925| 47| 76|123|
 2155|2011-04-04 00:00:00|summer| 0| 4| 15| 0| 1| 1| Clear|0.76|0.6667|0.23| 0.5522| 47| 96|143|
 2156|2011-04-04 00:00:00|summer| 0| 4| 16| 0| 1| 1| Clear|0.76|0.6515|0.22| 0.5224| 59| 130|189|
 2157|2011-04-04 00:00:00|summer| 0| 4| 17| 0| 1| 1| Clear|0.74|0.6515|0.23| 0.6119| 83| 283|366|
 2158|2011-04-04 00:00:00|summer| 0| 4| 18| 0| 1| 1| Mist|0.72|0.6364|0.23| 0.4925| 78| 308|386|
 2159|2011-04-04 00:00:00|summer| 0| 4| 19| 0| 1| 1| Clear| 0.7|0.6364|0.24| 0.4627| 51| 227|278|
 2160|2011-04-04 00:00:00|summer| 0| 4| 20| 0| 1| 1| Mist| 0.7|0.6364|0.24| 0.5522| 40| 133|173|
 2161|2011-04-04 00:00:00|summer| 0| 4| 21| 0| 1| 1| Mist| 0.7|0.6364| 0.3| 0.4478| 19| 76| 95|
 2162|2011-04-04 00:00:00|summer| 0| 4| 22| 0| 1| 1| Mist|0.68|0.6364|0.36| 0.4627| 17| 58| 75|
+-------+-------------------+------+---+----+---+-------+-------+----------+---------------+----+------+----+---------+------+----------+---+
only showing top 20 rows

In [0]:
# maximum booking in each season group in the newDF .
max_season_df = sqlContext.sql('select season, max(cnt) from bike_prediction_table group by season')

In [0]:
max_season_df.select("max(cnt)").show()

+--------+
max(cnt)|
+--------+
 967|
 957|
 801|
 977|
+--------+

# DataPreProcessing

## Preprocessing the data

So what do we need to do to get our data ready for Machine Learning?

**Recall our goal**: We want to learn to predict the count of bike rentals (the `cnt` column).  We refer to the count as our target "label".

**Features**: What can we use as features to predict the `cnt` label?  

All the columns except `cnt`, and a few exceptions:
* `casual` & `registered`
  * The `cnt` column we want to predict equals the sum of the `casual` + `registered` columns.  We will remove the `casual` and `registered` columns from the data to make sure we do not use them to predict `cnt`.  (**Warning: This is a danger in careless Machine Learning.  Make sure you do not "cheat" by using information you will not have when making predictions**)
* `season` and the date column `dteday`: We could keep them, but they are well-represented by the other date-related columns like `yr`, `mnth`, and `weekday`.
* `holiday` and `weekday`: These features are highly correlated with the `workingday` column.
* row index column `instant`: This is a useless column to us.

## Data Cleaning

### Handling Missing Data

1. Filter Out 
2. Fill In

In [0]:
%sh 
cat > movies.csv <<EOF 
name,rating,studio,date 
Avengers Endgame, 5, marvel,1260759144
Batman Vs Superman,4 ,DC,835355664
The Joker ,, DC,835355681
Frozen,4,Disney,835355604
Hitman,,,
EOF 


In [0]:
%sh ls -alh  /databricks/driver/movies.csv


-rw-r--r-- 1 root root 161 Oct 17 16:52 /databricks/driver/movies.csv

In [0]:
path_to_file = "file:///databricks/driver/movies.csv"

moviesDF = (spark.read          # Our DataFrameReader
  .option("header", "true")      # Let Spark know we have a header
  .option("inferSchema", "true") # Infering the schema (it is a small dataset)
  .csv(path_to_file)                 # Location of our data
  .cache()                       # Mark the DataFrame as cached.
)


In [0]:
moviesDF.printSchema()

root
-- name: string (nullable = true)
-- rating: double (nullable = true)
-- studio: string (nullable = true)
-- date : integer (nullable = true)

In [0]:
moviesDF.show()

+------------------+------+-------+----------+
 name|rating| studio| date |
+------------------+------+-------+----------+
 Avengers Endgame| 5.0| marvel|1260759144|
Batman Vs Superman| 4.0| DC| 835355664|
 The Joker | null| DC| 835355681|
 Frozen| 4.0| Disney| 835355604|
 Hitman| null| null| null|
+------------------+------+-------+----------+

#### 1. Filter OUT

[DROP](https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dropna)

In [0]:
# DROP Any Row with Any NULL Valu
moviesDF.na.drop().show()

+------------------+------+-------+----------+
 name|rating| studio| date |
+------------------+------+-------+----------+
 Avengers Endgame| 5.0| marvel|1260759144|
Batman Vs Superman| 4.0| DC| 835355664|
 Frozen| 4.0| Disney| 835355604|
+------------------+------+-------+----------+

In [0]:
moviesDF.na.drop("all").show()
moviesDF.na.drop("any").show()

+------------------+------+-------+----------+
 name|rating| studio| date |
+------------------+------+-------+----------+
 Avengers Endgame| 5.0| marvel|1260759144|
Batman Vs Superman| 4.0| DC| 835355664|
 The Joker | null| DC| 835355681|
 Frozen| 4.0| Disney| 835355604|
 Hitman| null| null| null|
+------------------+------+-------+----------+

+------------------+------+-------+----------+
 name|rating| studio| date |
+------------------+------+-------+----------+
 Avengers Endgame| 5.0| marvel|1260759144|
Batman Vs Superman| 4.0| DC| 835355664|
 Frozen| 4.0| Disney| 835355604|
+------------------+------+-------+----------+

In [0]:
## APPLY to Specific Column
moviesDF.na.drop("all", subset = ['studio','rating']).show()

+------------------+------+-------+----------+
 name|rating| studio| date |
+------------------+------+-------+----------+
 Avengers Endgame| 5.0| marvel|1260759144|
Batman Vs Superman| 4.0| DC| 835355664|
 The Joker | null| DC| 835355681|
 Frozen| 4.0| Disney| 835355604|
+------------------+------+-------+----------+

#### 2. Fill In

In [0]:
## TYPE Inference of INT in DataFrames

moviesDF.na.fill("1").show()
moviesDF.na.fill(835355604).show()
moviesDF.na.fill("OTHER STUDIO").show()
moviesDF.na.fill({'rating':0.0})

+------------------+------+-------+----------+
 name|rating| studio| date |
+------------------+------+-------+----------+
 Avengers Endgame| 5.0| marvel|1260759144|
Batman Vs Superman| 4.0| DC| 835355664|
 The Joker | null| DC| 835355681|
 Frozen| 4.0| Disney| 835355604|
 Hitman| null| 1| null|
+------------------+------+-------+----------+

+------------------+------------+-------+----------+
 name| rating| studio| date |
+------------------+------------+-------+----------+
 Avengers Endgame| 5.0| marvel|1260759144|
Batman Vs Superman| 4.0| DC| 835355664|
 The Joker |8.35355604E8| DC| 835355681|
 Frozen| 4.0| Disney| 835355604|
 Hitman|8.35355604E8| null| 835355604|
+------------------+------------+-------+----------+

+------------------+------+------------+----------+
 name|rating| studio| date |
+------------------+------+------------+----------+
 Avengers Endgame| 5.0| marvel|1260759144|
Batman Vs Superman| 4.0| DC| 835355664|
 The Joker | null| DC| 835355681|
 Frozen| 4.0| Disney| 835355604|
 Hitman| null|OTHER STUDIO| null|
+------------------+------+------------+----------+

Out[50]: DataFrame[name: string, rating: double, studio: string, date : int]

In [0]:
## NOTE : FOR PRODUCTION always use the names
moviesDF.select("rating").describe().show()
moviesDF.select(avg(moviesDF['rating'] )).show()
moviesDF.fillna(4.3, subset=['rating']).show()
moviesDF.na.fill({'rating':0.0})

+-------+------------------+
summary| rating|
+-------+------------------+
 count| 3|
 mean| 4.333333333333333|
 stddev|0.5773502691896258|
 min| 4.0|
 max| 5.0|
+-------+------------------+

+-----------------+
 avg(rating)|
+-----------------+
4.333333333333333|
+-----------------+

+------------------+------+-------+----------+
 name|rating| studio| date |
+------------------+------+-------+----------+
 Avengers Endgame| 5.0| marvel|1260759144|
Batman Vs Superman| 4.0| DC| 835355664|
 The Joker | 4.3| DC| 835355681|
 Frozen| 4.0| Disney| 835355604|
 Hitman| 4.3| null| null|
+------------------+------+-------+----------+

Out[51]: DataFrame[name: string, rating: double, studio: string, date : int]

### Remove Trivial Data

Let's drop the columns `instant`, `dteday`, `season`, `casual`, `holiday`, `weekday`, and `registered` from our DataFrame and then review our schema:

In [0]:
preprocessedDF = initialDF.drop("instant", "dteday", "season", "casual", "registered", "holiday", "weekday")

preprocessedDF.printSchema()

root
-- yr: integer (nullable = true)
-- mnth: integer (nullable = true)
-- hr: integer (nullable = true)
-- workingday: integer (nullable = true)
-- weathersit: integer (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- hum: double (nullable = true)
-- windspeed: double (nullable = true)
-- cnt: integer (nullable = true)

### dropDuplicates Columns

In [0]:
newDF.select('Season','windspeed','atemp').count()

Out[53]: 17379

In [0]:
newDF.select('Season','windspeed','atemp').dropDuplicates().count()


Out[54]: 1994

In [0]:
newDF.select('Season','windspeed','atemp').dropDuplicates().show()

+------+---------+------+
Season|windspeed| atemp|
+------+---------+------+
spring| 0.2239|0.4091|
spring| 0.2537|0.4242|
spring| 0.2537|0.4394|
spring| 0.0|0.2879|
spring| 0.2985|0.4242|
spring| 0.2239|0.3939|
spring| 0.2985|0.4394|
spring| 0.2537|0.4091|
spring| 0.2985|0.4545|
spring| 0.2836|0.4394|
spring| 0.2836|0.3333|
spring| 0.0|0.3485|
spring| 0.2537|0.3939|
spring| 0.0896|0.2576|
spring| 0.2239|0.3485|
spring| 0.2836|0.4545|
spring| 0.194|0.4545|
spring| 0.194|0.4091|
spring| 0.2836|0.4242|
spring| 0.0|0.2576|
+------+---------+------+
only showing top 20 rows

In [0]:
newDF.select("Season").distinct().show()

+------+
Season|
+------+
winter|
summer|
spring|
 fall|
+------+